In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from small_script.myFunctions import *
import feather
import Bio.PDB as bio

from sklearn.metrics import confusion_matrix


d3_to_index = bio.Polypeptide.d3_to_index  # we may want to adjust this in the future.
three_to_one = bio.Polypeptide.three_to_one
one_to_index = bio.Polypeptide.one_to_index
plt.rcParams['figure.figsize'] = [16.18033, 10]

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [8]:
data = feather.read_dataframe("/Users/weilu/Research/optimization/fragment/feather_cluster_data.feather")

In [15]:
data.head()

,pdb,i,seq,dd,category,count,s1,s2,s3,s4,s5,s6,s7,s8,s9
0,1igqB00,0,DKLKKAIVQ,"9,13,11,15,15,19,9,9,13,15,17,5,9,11,15,9,13,1...",-1,-1,2,8,9,8,8,0,7,17,13
1,1igqB00,1,KLKKAIVQV,"9,9,13,15,17,21,5,9,11,15,17,9,13,15,19,9,13,1...",-1,-1,8,9,8,8,0,7,17,13,17
2,1igqB00,2,LKKAIVQVE,"5,9,11,15,17,21,9,13,15,19,23,9,13,17,19,9,13,...",-1,-1,9,8,8,0,7,17,13,17,3
3,1igqB00,3,KKAIVQVEH,"9,13,15,19,23,25,9,13,17,19,21,9,13,17,19,9,13...",9987,13,8,8,0,7,17,13,17,3,6
4,1igqB00,4,KAIVQVEHD,"9,13,17,19,21,25,9,13,17,19,23,9,13,15,19,11,1...",6835,18,8,0,7,17,13,17,3,6,2


In [56]:
data["dd"].value_counts()

5,7,9,9,11,13,5,7,9,9,11,5,7,9,9,5,7,9,5,7,5                    39130
5,7,9,11,11,13,5,7,9,9,11,5,7,9,9,5,7,9,5,7,5                   20575
5,7,9,9,11,13,5,7,9,9,11,5,7,9,11,5,7,9,5,7,5                   18996
5,7,9,9,11,13,5,7,9,11,11,5,7,9,9,5,7,9,5,7,5                   18382
5,7,9,11,11,13,5,7,9,9,11,5,7,9,11,5,7,9,5,7,5                  14304
5,7,9,9,11,13,5,7,9,11,11,5,7,9,11,5,7,9,5,7,5                  12961
5,7,9,11,11,13,5,7,9,11,11,5,7,9,9,5,7,9,5,7,5                  12641
5,7,9,11,11,13,5,7,9,11,11,5,7,9,11,5,7,9,5,7,5                 12033
5,7,9,9,11,13,5,7,9,9,11,5,5,9,9,5,7,9,5,7,5                     7326
5,7,9,9,11,13,5,7,9,11,11,5,5,9,9,5,7,9,5,7,5                    6656
5,7,9,11,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5                    5374
5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5                     5195
5,7,9,9,11,13,5,7,9,9,11,5,7,9,9,5,5,9,5,7,5                     4941
5,7,9,9,11,13,5,7,9,9,11,5,7,9,9,5,7,9,5,5,5                     4867
5,7,9,9,11,13,5,7,9,

In [10]:
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb


Using TensorFlow backend.


In [19]:
x_train.shape

(25000, 100)

In [40]:
data.query("category == 11").shape

(5195, 15)

# cat 10 and 11 are very similar. they should be in the same group.

In [59]:
data.query("category == 10 or category == 11").sample(10)

,pdb,i,seq,dd,category,count,s1,s2,s3,s4,s5,s6,s7,s8,s9
299936,3tulA00,66,AKSVYDAAT,"5,7,9,11,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",10,5374,0,8,15,17,19,2,0,0,16
1876837,1k1fA00,27,DIEQELERA,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,2,7,3,13,3,9,3,14,0
418011,1ztpA01,107,AWAGIARAV,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,0,18,0,5,7,0,14,0,17
1211494,3zkvA00,758,QFLTHFVMQ,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,13,4,9,16,6,4,17,10,13
313392,1pp1X01,110,HCCSLLIGV,"5,7,9,11,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",10,5374,6,1,1,15,9,9,7,5,17
429159,2j7qA00,169,DFTEAISAL,"5,7,9,11,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",10,5374,2,4,16,3,0,7,15,0,9
1323194,1s7oB00,96,KISILTSID,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,8,7,15,7,9,16,15,7,2
192023,2pftA00,27,YLGSMAKIQ,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,19,9,5,15,10,0,8,7,13
1204807,4f3qA01,7,AKARQDAKR,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,0,8,0,14,13,2,0,8,14
1324321,3aqpA02,21,LEKARTVLE,"5,7,9,11,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",10,5374,9,3,8,0,14,16,17,9,3


In [58]:
data.query("category == 29 or category == 11").sample(10)

,pdb,i,seq,dd,category,count,s1,s2,s3,s4,s5,s6,s7,s8,s9
1211147,3zkvA00,395,LEACIYSFQ,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,9,3,0,1,7,19,15,4,13
1711744,2itbA00,36,FKAASTALS,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,4,8,0,0,15,16,0,9,15
413878,3rjvA01,15,RAQYYLADT,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,14,0,13,19,19,9,0,2,16
1836419,5tkyA03,30,ALRRLRTAC,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,0,9,14,14,9,14,16,0,1
1216811,1w9cA00,141,FFLLLQAVN,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,4,4,9,9,9,13,0,17,11
249112,1ej6B00,233,INPTEIEWA,"9,7,7,11,11,11,5,7,9,11,11,5,7,9,9,5,7,9,5,7,5",29,2452,7,11,12,16,3,7,3,18,0
1664651,2qtfA01,42,IQYDKLQQI,"9,7,7,11,11,11,5,7,9,11,11,5,7,9,9,5,7,9,5,7,5",29,2452,7,13,19,2,8,9,13,13,7
950129,1xflA00,46,APFFADLAK,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,0,12,4,4,0,2,9,0,8
310748,2yyuB00,138,VETVAHYAA,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,17,3,16,17,0,6,19,0,0
822123,3qvsA01,236,PLILDIARF,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,12,9,7,9,2,7,0,14,4


In [55]:
data.query("category == 4 or category == 11").sample(10)

,pdb,i,seq,dd,category,count,s1,s2,s3,s4,s5,s6,s7,s8,s9
1691660,3ar4A04,179,TGPVKEKIL,"5,7,9,11,11,13,5,7,9,9,11,5,7,9,11,5,7,9,5,7,5",4,14304,16,5,12,17,8,3,8,7,9
606678,4fguB02,2,DVPLTIMKR,"5,7,9,11,11,13,5,7,9,9,11,5,7,9,11,5,7,9,5,7,5",4,14304,2,17,12,9,16,7,10,8,14
1297613,4muoA02,229,GLARVNQAF,"5,7,9,11,11,13,5,7,9,9,11,5,7,9,11,5,7,9,5,7,5",4,14304,5,9,0,14,17,11,13,0,4
395061,5aj3P00,95,TNAERLRRK,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,16,11,0,3,14,9,14,14,8
1029953,2oryA00,207,NADFADYFD,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,11,0,2,4,0,2,19,4,2
1765556,1zhcA00,45,VSHMKKQKL,"5,7,9,11,11,13,5,7,9,9,11,5,7,9,11,5,7,9,5,7,5",4,14304,17,15,6,10,8,8,13,8,9
755750,4i9cA02,181,DLEDFAIDV,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,2,9,3,2,4,0,7,2,17
1486275,1h9eA00,14,LKSELVANN,"5,7,9,11,11,13,5,7,9,9,11,5,7,9,11,5,7,9,5,7,5",4,14304,9,8,15,3,9,17,0,11,11
1200946,3s1sA02,125,DIELGKVLS,"5,7,9,11,11,13,5,7,9,9,11,5,7,9,11,5,7,9,5,7,5",4,14304,2,7,3,9,5,8,17,9,15
1072383,3it4B01,56,AQADLDEAV,"5,7,9,11,11,13,5,7,9,9,11,5,7,9,11,5,7,9,5,7,5",4,14304,0,13,0,2,9,2,3,0,17


In [28]:
test.head()

,pdb,i,seq,dd,category,count,s1,s2,s3,s4,s5,s6,s7,s8,s9
929058,3i4rB00,40,DRAVTQISV,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,2,14,0,17,16,13,7,15,17
1276353,4tl2A02,72,DWHDLAAFW,"5,7,9,11,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",10,5374,2,18,6,2,9,0,0,4,18
869288,1xdpA01,31,GIYSNNLDE,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,5,7,19,15,11,11,9,2,3
1358821,2g7lA00,132,LQDATATAA,"5,7,9,11,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",10,5374,9,13,2,0,16,0,16,0,0
857911,2jbrA03,83,ARALLEKTW,"5,7,9,9,11,13,5,5,9,9,11,5,7,9,9,5,7,9,5,7,5",11,5195,0,14,0,9,9,3,8,16,18


In [102]:
data.query("category == 9").shape

(6656, 15)

In [ ]:
data.sample(replace)

In [117]:
test = data.query("category > -1 and category < 10").groupby("category").apply(pd.DataFrame.sample, 10000, replace=True)
# test = data.query("category > -1 and category < 10").sample(10000)
x_train = test.iloc[:, 6:15].values
y_train_value = test["category"].values
test = data.query("category > -1 and category < 10").groupby("category").apply(pd.DataFrame.sample, 10000, replace=True)
# test = data.query("category > -1 and category < 10").sample(10000)
x_test = test.iloc[:, 6:15].values
y_test_value = test["category"].values

# print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
y_train = to_categorical(np.array(y_train_value))
y_test = to_categorical(np.array(y_test_value))

Pad sequences (samples x time)
x_train shape: (100000, 9)


In [118]:
y_train.sum(axis=0)

array([10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000.], dtype=float32)

In [ ]:
from keras.utils import to_categorical
max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 9
batch_size = 32

print(len(x_train), 'train sequences')



model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# try using different optimizers and different optimizer configs
# model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=[x_test, y_test])

100000 train sequences
Train...
Train on 100000 samples, validate on 100000 samples
Epoch 1/10
 29376/100000 [=======>......................] - ETA: 1:20 - loss: 2.2891 - acc: 0.1209

In [112]:
y_pred = model.predict(x_test)
predicted = np.argmax(y_pred, axis=1)
confusion_matrix(y_test_value, predicted)

array([[ 97,  94,  98,  93,  67, 119,  71, 100, 207,  54],
       [ 71, 105,  84,  73, 126, 107,  87, 120, 175,  52],
       [ 71,  87, 100,  68,  98, 133,  58, 110, 208,  67],
       [ 68,  83,  91, 125,  75, 151,  75, 103, 164,  65],
       [ 57, 100,  99,  74, 116, 119,  86, 138, 150,  61],
       [ 61,  61, 104, 100,  73, 161,  79, 125, 177,  59],
       [ 58,  99,  78, 103, 119, 120, 109, 132, 141,  41],
       [ 62,  85,  92,  87, 119, 142,  83, 130, 137,  63],
       [ 80,  67,  98,  60,  76, 117,  39,  93, 306,  64],
       [ 67,  53, 120,  86,  70, 142,  52,  90, 218, 102]])

In [107]:
y_pred = model.predict(x_test)

In [108]:
predicted = np.argmax(y_pred, axis=1)

In [110]:
confusion_matrix(y_test_value, predicted)

array([[107, 102, 141, 114,  57, 104,  74,  73, 138,  90],
       [ 82, 123,  97,  81, 108, 102, 100, 111, 118,  78],
       [ 88, 102, 133,  79,  88, 129,  48,  93, 123, 117],
       [ 81,  91, 122, 133,  68, 143,  87,  77,  97, 101],
       [ 66, 121, 128,  87, 114, 109,  80, 116, 105,  74],
       [ 89,  87, 115, 132,  72, 137,  71,  88, 109, 100],
       [ 70, 125,  94, 114, 113, 115, 107, 106,  89,  67],
       [ 64, 102, 119, 110, 105, 131,  94, 109,  86,  80],
       [ 96,  86, 124,  69,  66, 103,  40,  77, 219, 120],
       [ 82,  71, 126, 110,  61, 140,  45,  85, 128, 152]])

In [79]:
y_test[-1]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [96]:
y_test.sum(axis=0)

array([2327., 1264., 1173., 1126.,  918.,  839.,  789.,  724.,  446.,
        394.], dtype=float32)

In [72]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [48]:
max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 9
batch_size = 32

print(len(x_train), 'train sequences')


print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
print('x_train shape:', x_train.shape)
y_train = np.array(y_train)

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=40,
          validation_data=[x_train, y_train])

1000 train sequences
Pad sequences (samples x time)
x_train shape: (1000, 9)
Train...
Train on 1000 samples, validate on 1000 samples
Epoch 1/40
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6922 - acc: 0.5350 - val_loss: 0.6884 - val_acc: 0.5350
Epoch 2/40
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6875 - acc: 0.5500 - val_loss: 0.6816 - val_acc: 0.5980
Epoch 3/40
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6810 - acc: 0.5840 - val_loss: 0.6734 - val_acc: 0.5810
Epoch 4/40
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6768 - acc: 0.5630 - val_loss: 0.6698 - val_acc: 0.6030
Epoch 5/40
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6679 - acc: 0.5940 - val_loss: 0.6592 - val_acc: 0.6030
Epoch 6/40
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6623 - acc: 0.6020 - val_loss: 0.6546 - val_acc: 0.6100
Epoch 7/40
1000/1000 [==============================] - 1s 1ms

In [11]:
max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[x_test, y_test])

Loading data...
17465344/17464789 [==============================] - 2s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 98s 4ms/step - loss: 0.4393 - acc: 0.7982 - val_loss: 0.3589 - val_acc: 0.8412
Epoch 2/4
25000/25000 [==============================] - 99s 4ms/step - loss: 0.2482 - acc: 0.9024 - val_loss: 0.4077 - val_acc: 0.8422
Epoch 3/4
25000/25000 [==============================] - 99s 4ms/step - loss: 0.1685 - acc: 0.9390 - val_loss: 0.4364 - val_acc: 0.8392
Epoch 4/4
25000/25000 [==============================] - 98s 4ms/step - loss: 0.1138 - acc: 0.9591 - val_loss: 0.5522 - val_acc: 0.8333


In [17]:
data.head()

,pdb,i,seq,dd,category,count,s1,s2,s3,s4,s5,s6,s7,s8,s9
0,1igqB00,0,DKLKKAIVQ,"9,13,11,15,15,19,9,9,13,15,17,5,9,11,15,9,13,1...",-1,-1,2,8,9,8,8,0,7,17,13
1,1igqB00,1,KLKKAIVQV,"9,9,13,15,17,21,5,9,11,15,17,9,13,15,19,9,13,1...",-1,-1,8,9,8,8,0,7,17,13,17
2,1igqB00,2,LKKAIVQVE,"5,9,11,15,17,21,9,13,15,19,23,9,13,17,19,9,13,...",-1,-1,9,8,8,0,7,17,13,17,3
3,1igqB00,3,KKAIVQVEH,"9,13,15,19,23,25,9,13,17,19,21,9,13,17,19,9,13...",9987,13,8,8,0,7,17,13,17,3,6
4,1igqB00,4,KAIVQVEHD,"9,13,17,19,21,25,9,13,17,19,23,9,13,15,19,11,1...",6835,18,8,0,7,17,13,17,3,6,2
